In [54]:
#Start the Bloomberg Console
%run pdblp.ipynb  #Load the functions from PDBLP. This contains access to the BDP and BDH functionality with a Pandas layer on top

con = BCon(debug=True, port=8194)
con.start()
con.debug = False

In [55]:
'''deliverable basket for TYZ9'''
''' pull results from bloomberg SRCH'''

my_srch = con.bsrch("FI:TYU9 DELIV")
my_srch = my_srch.values.tolist()
print(my_srch)


[['912828Y9 Govt'], ['9128286B Govt'], ['9128285M Govt'], ['9128284V Govt'], ['9128282A Govt'], ['9128287B Govt'], ['9128286S Govt'], ['912828YD Govt'], ['9128282R Govt'], ['9128283F Govt'], ['9128283W Govt'], ['9128284N Govt'], ['912828P4 Govt'], ['912828U2 Govt'], ['912828V9 Govt'], ['912828R3 Govt'], ['912828X8 Govt'], ['912828M5 Govt'], ['9128286X Govt'], ['9128285T Govt'], ['9128285J Govt'], ['9128286A Govt'], ['9128285C Govt'], ['9128286F Govt'], ['9128286L Govt'], ['9128285N Govt']]


In [56]:
'''example of getting standard settle date'''


security = 'T3.125 11/15/2028 Govt'

trd_dt = '20190904'
set_date = con.ref(security, 'SETTLE_DT',[('USER_LOCAL_TRADE_DATE',trd_dt)]).value[0]
date = set_date.strftime('%Y%m%d')
print(date)

20190905


In [57]:
'''print description'''

trd_dt = '20190903'
descrip = con.ref('9128285M Govt', 'SECURITY_DES').value[0]

print(descrip)

T 3 1/8 11/15/28


In [58]:
'''SOFR Interpolation to futures delivery date'''

s_date = '20190815'   # start date to run interpolation
e_date = '20190904'   # end date to run interpolation
delivery_date = '20191202' # futures devliery date for repo

one_wk= 7
two_wk = 14
one_month = 30
two_month = 60
three_month = 90
four_month = 120
five_month = 150
six_month = 180
seven_month = 210
eight_month = 240
nine_month = 270

sofr_tickers = {7 : 'USOSFR1Z Curncy', 14 : 'USOSFR2Z Curncy', 30 : 'USOSFRA Curncy', 60 : 'USOSFRB Curncy', 
               90 : 'USOSFRC Curncy' , 120 : 'USOSFRD Curncy' , 150 : 'USOSFRE Curncy' , 180 : 'USOSFRF Curncy',
               210 : 'USOSFRG Curncy' , 240 : 'USOSFRH Curncy', 270 : 'USOSFRI Curncy'}



time_periods = [one_wk, two_wk, one_month, two_month, three_month, four_month, five_month, 
                six_month, seven_month, eight_month, nine_month]

def date_range():
    df = con.bdh('USOSFRA Curncy', ['PX_LAST'], s_date, e_date, elms=[("calendarCodeOverride", "US")])
    t_date =df.index
    return t_date.strftime('%Y%m%d')  #date list in bloomberg format



interp_sofr = []   # create empty list

def low(x):
    
    time = datetime.strptime(delivery_date,'%Y%m%d')- datetime.strptime(x,'%Y%m%d')
    time = time.days # calculate days to delivery date
    
    z = time_periods
    
    for e, i in enumerate(z):
        if i < time:
            continue
        else:
            n = z[e-1]
            ld = con.bdh(sofr_tickers[n], ['PX_LAST'], x, x)
            ld = ld.unstack()
            ld = ld[0]
            ud = con.bdh(sofr_tickers[i], ['PX_LAST'], x, x)
            ud = ud.unstack()
            ud = ud[0]
            interpolated_rate = ((time - n)/(i-n) * (ud - ld)) + ld
            return x ,  '%.4f' % interpolated_rate
            break
        
        
def all_dates(d):
    for i in d:
        interp = low(i)
        interp_sofr.append(interp)
        
print(all_dates(date_range()))

'''Interpolated SOFR to CSV'''
print(interp_sofr)

I_sofr= pd.DataFrame(interp_sofr)
I_sofr.columns = ['Date','Rate']
I_sofr['Date'] = pd.DatetimeIndex(I_sofr['Date'])
I_sofr['Rate'] = pd.to_numeric(I_sofr['Rate'])
#I_sofr.to_csv('I_sofr.csv',mode='a',header=None)
I_sofr.to_csv('I_sofr.csv')  # will overwrite file



None
[('20190815', '1.9166'), ('20190816', '1.9350'), ('20190819', '1.9568'), ('20190820', '2.2221'), ('20190821', '1.9564'), ('20190822', '1.9590'), ('20190823', '1.9200'), ('20190826', '1.9380'), ('20190827', '1.9370'), ('20190828', '1.9400'), ('20190829', '1.9210'), ('20190830', '1.9190'), ('20190903', '1.8770'), ('20190904', '1.9303')]


In [59]:
''' FUTURES PRICER'''

contract = 'TYZ9 Comdty'
start_dt = '20190815'
end_dt = '20190904'
delivery_date = "20191202"

fut_output=[]

def date_range():
    df = con.bdh("T 3 09/30/2025 Govt", ['BB_LIBOR_MMS_OIS_DISC_MID'], start_dt, end_dt)
    x =df.index
    return x.strftime('%Y%m%d')  #date list in bloomberg format



def pricer(x): 
    trd_dt = x
    ctd = con.bdh(contract, ['FUT_CTD_CUSIP'], trd_dt, trd_dt)
    ctd = ctd.unstack()
    ctd = ctd[0]
    ctd = ctd + " Govt"
    ctd_price = con.bdh(ctd, ['PX_LAST'], trd_dt, trd_dt)
    ctd_price = ctd_price.unstack()
    ctd_price = ctd_price[0]
    gross_basis = con.bdh(contract, ['FUT_CTD_GROSS_BASIS'], trd_dt, trd_dt)
    gross_basis = gross_basis.unstack()
    gross_basis = gross_basis[0]/32
    cash_fwd_prc = ctd_price - gross_basis
    fwd_yld = con.ref(ctd, 'YAS_BOND_YLD',[('YAS_BOND_PX',cash_fwd_prc),('SETTLE_DT',delivery_date)]).value[0]
    fwd_z = con.ref(ctd, 'YAS_ZSPREAD',[('YAS_BOND_YLD',fwd_yld),('OAS_CURVE_DT', trd_dt),('SETTLE_DT',delivery_date)]).value[0]
    return (trd_dt, delivery_date, ".TYA Corp", contract,"NA", cash_fwd_prc,"NA" ,'%.3f' % fwd_yld, '%.3f' % fwd_z)

def one_security_loop():
    x = date_range()
    for i in x:
        fut_out = pricer(i)
        print(fut_out)
        fut_output.append(fut_out)

print(one_security_loop())

('20190815', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.704003125, 'NA', '1.443', '11.196')
('20190816', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.64829374999999, 'NA', '1.456', '10.619')
('20190819', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.21888125, 'NA', '1.523', '11.215')
('20190820', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.556675, 'NA', '1.466', '10.151')
('20190821', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.335565625, 'NA', '1.505', '10.364')
('20190822', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.16819375, 'NA', '1.531', '9.146')
('20190823', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.677096875, 'NA', '1.452', '9.940')
('20190826', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.66791875, 'NA', '1.453', '9.793')
('20190827', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.987590625, 'NA', '1.398', '10.314')
('20190828', '20191202', '.TYA Corp', 'TYZ9 Comdty', 'NA', 102.970803125, 'NA', '1.401', '11.558')
('20190829', '